# SQL Q&A
https://python.langchain.com/docs/tutorials/sql_qa/#agents

In [1]:
# Make sure to set your env varaiable
# export LANGSMITH_API_KEY=[API KEY]

from getpass import getpass
import os

if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass()
    os.environ["LANGSMITH_API_KEY"] = 'true'
    


In [2]:
from langchain_community.utilities import SQLDatabase

from toelo.player_elo.database_connection import get_connection_string, DATABASE_CONFIG


db_uri = get_connection_string(DATABASE_CONFIG)
db = SQLDatabase.from_uri(db_uri)
db.run("SELECT * FROM players_elo LIMIT 5;")

"[(33042, 2014, 'Migjen', 'Basha', 'Migjen Basha', 'migjen-basha', 'Switzerland', datetime.date(1987, 1, 5), 2593.2273077268505), (159029, 2013, 'Hüseyin', 'Kala', 'Hüseyin Kala', 'huseyin-kala', 'Turkey', datetime.date(1987, 5, 5), 2536.2046353257374), (167061, 2012, 'Leighton', 'McIntosh', 'Leighton McIntosh', 'leighton-mcintosh', 'Scotland', datetime.date(1993, 2, 6), 2255.7503253540335), (22598, 2014, 'Gianni', 'Munari', 'Gianni Munari', 'gianni-munari', 'Italy', datetime.date(1983, 6, 24), 2574.7604517201576), (118056, 2014, 'Mavroudis', 'Bougaidis', 'Mavroudis Bougaidis', 'mavroudis-bougaidis', 'Greece', datetime.date(1993, 6, 1), 2507.888919411905)]"

In [ ]:
from pathlib import Path
from typing_extensions import TypedDict

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


from langchain_community.llms import GPT4All

# Chat model
llm_path = Path.home() / Path("models") / "mistral-7b-openorca.Q4_0.gguf"
llm = GPT4All(model=str(llm_path), n_threads=8)

# https://python.langchain.com/docs/tutorials/sql_qa/#agents